In [6]:
import os
import mysql.connector
import pandas as pd
import sqlalchemy


from dotenv import load_dotenv
load_dotenv()


USERNAME = os.getenv('USER1')
PASSWORD = os.getenv('PASSWORD1')
HOST = os.getenv('HOST1')
DATABASE = os.getenv('DATABASE1')


conn_str = 'mysql+mysqlconnector://{0}:{1}@{2}/{3}'.format(USERNAME, PASSWORD, HOST, DATABASE)

conn = sqlalchemy.create_engine(conn_str)

%config SqlMagic.autopandas=True
%config SqlMagic.displaycon=False
%config SqlMagic.feedback=False
%reload_ext sql
%sql {conn_str}

What if you wanted to figure out how many incidents get reported on each day of the week? Better yet, what if you wanted to know how many incidents happen, on average, on a Friday in December? In January? There are two steps to this process: counting the number of incidents each day (inner query), then determining the monthly average (outer query):

In [ ]:
%%sql
    select day_of_week,date,count(incidnt_num) as incidents
    from sf
    group by 1,2
    limit 10

In [ ]:
%%sql
select sf.* 
from sf join (select distinct day_of_week from sf where day_of_week like "%day") dow limit 10

In [ ]:
%%sql
select sf.*
from sf join (select distinct day_of_week from sf where day_of_week like "%day") dow on sf.day_of_week=dow.day_of_week
limit 10

In [7]:
%%sql
   select day_of_week,date,count(incidnt_num) as incidents
    from (
select sf.*
from sf join (select distinct day_of_week from sf where day_of_week like "%day") dow on sf.day_of_week=dow.day_of_week
    ) tb1
    group by 1,2
    limit 10

,day_of_week,date,incidents
0,Friday,2014-01-31 08:00:00,217
1,Thursday,2014-01-30 08:00:00,248
2,Wednesday,2014-01-29 08:00:00,225
3,Tuesday,2014-01-28 08:00:00,225
4,Monday,2014-01-27 08:00:00,261
5,Sunday,2014-01-26 08:00:00,248
6,Wednesday,2013-12-04 08:00:00,235
7,Wednesday,2013-12-11 08:00:00,248
8,Wednesday,2013-12-18 08:00:00,249
9,Wednesday,2013-12-25 08:00:00,91


In [10]:
%%sql
select substr(date, 6, 2) as month,
    day_of_week,
    avg(incidents) as average_incidnts
from (
        select day_of_week,
            date,
            count(incidnt_num) as incidents
        from (
                select sf.*
                from sf
                    join (
                        select distinct day_of_week
                        from sf
                        where day_of_week like "%day"
                    ) dow on sf.day_of_week = dow.day_of_week
            ) tb1
        group by 1, 2
    ) tb2
    group by 1,2
limit 20

,month,day_of_week,average_incidnts
0,01,Friday,267.4000
1,01,Thursday,256.0000
2,01,Wednesday,257.2000
3,01,Tuesday,241.7500
4,01,Monday,250.0000
5,01,Sunday,236.2500
6,01,Saturday,258.7500
7,12,Tuesday,232.0000
8,12,Monday,222.8000
9,12,Sunday,241.2000


In [18]:
 %%sql
 select date from sf where date > 0

,date
0,2014-01-31 08:00:00
1,2014-01-31 08:00:00
2,2014-01-31 08:00:00
3,2014-01-31 08:00:00
4,2014-01-31 08:00:00
...,...
23222,2013-11-01 07:00:00
23223,2013-11-01 07:00:00
23224,2013-11-01 07:00:00
23225,2013-11-01 07:00:00


In [19]:
%%sql
select * 
from sf
where date= ( select min(date) from sf where date > 0)


,incidnt_num,category,descript,day_of_week,date,time,pd_district,resolution,address,lon,lat,location,id
0,130927532.0,OTHER OFFENSES,MISCELLANEOUS INVESTIGATION,Friday,2013-11-01 07:00:00,10:00,CENTRAL,NONE,600 Block of CLAY ST,-122.404290,37.794552,"""(37.7945521769103",-122
1,130927065.0,LARCENY/THEFT,GRAND THEFT FROM A BUILDING,Friday,2013-11-01 07:00:00,02:00,CENTRAL,"""ARREST","BOOKED""",300.000000,-122.405453,37.7900585196835,0
2,136209821.0,NON-CRIMINAL,LOST PROPERTY,Friday,2013-11-01 07:00:00,13:00,RICHMOND,NONE,3800 Block of CALIFORNIA ST,-122.457200,37.785966,"""(37.7859658707298",-122
3,130927065.0,LARCENY/THEFT,GRAND THEFT FROM PERSON,Friday,2013-11-01 07:00:00,02:00,CENTRAL,"""ARREST","BOOKED""",300.000000,-122.405453,37.7900585196835,0
4,130955997.0,NON-CRIMINAL,LOST PROPERTY,Friday,2013-11-01 07:00:00,23:30,SOUTHERN,NONE,1000 Block of FOLSOM ST,-122.405898,37.778235,"""(37.778235081987",-122
...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,130933216.0,NON-CRIMINAL,LOST PROPERTY,Friday,2013-11-01 07:00:00,16:00,CENTRAL,NONE,600 Block of BUSH ST,-122.407962,37.790297,"""(37.7902968724188",-122
398,130926374.0,PROSTITUTION,INDECENT EXPOSURE - PROSTITUTION RELATED,Friday,2013-11-01 07:00:00,00:15,MISSION,"""ARREST","CITED""",1300.000000,-122.403449,37.7531563892388,0
399,130965134.0,OTHER OFFENSES,FAILURE TO REGISTER AS SEX OFFENDER,Friday,2013-11-01 07:00:00,00:01,SOUTHERN,"""ARREST","BOOKED""",800.000000,-122.403675,37.7751775060199,0
400,136206623.0,LARCENY/THEFT,GRAND THEFT FROM UNLOCKED AUTO,Friday,2013-11-01 07:00:00,19:00,RICHMOND,NONE,CLAY ST / PIERCE ST,-122.437496,37.790291,"""(37.7902911486821",-122


In [20]:
%%sql
create view dc as
select * from dc_bikeshare_q1_2012_preprocessed

""


In [22]:
%%sql
select * from dc limit 1

,duration,duration_seconds,start_time,start_station,start_terminal,end_time,end_station,end_terminal,bike_number,rider_type,id
0,0h 19m 22sec.,1162.0,2012-01-01 00:10:00,Georgia & New Hampshire Ave NW,31400.0,2012-01-01 00:29:00,16th & Harvard St NW,31103.0,W00524,Casual,2


In [25]:
%%sql
select start_terminal, 
       duration_seconds,
    sum(duration_seconds) over (
        order by start_time
    ) as running_total
from dc
limit 10

,start_terminal,duration_seconds,running_total
0,31400.0,1162.0,2307.0
1,31400.0,1145.0,2307.0
2,31236.0,1754.0,4061.0
3,31214.0,1206.0,5267.0
4,31604.0,1124.0,6391.0
5,31300.0,1512.0,7903.0
6,31201.0,1212.0,10300.0
7,31201.0,1185.0,10300.0
8,31002.0,41427.0,52929.0
9,31002.0,1202.0,52929.0


In [27]:
%%sql
select start_terminal, 
       duration_seconds,
    sum(duration_seconds) over (
        partition by start_terminal
        order by start_time
    ) as running_total
from dc
where start_time < "2012-01-08"
limit 10

,start_terminal,duration_seconds,running_total
0,31000.0,1422.0,1422.0
1,31000.0,3340.0,4762.0
2,31000.0,2661.0,7423.0
3,31001.0,2876.0,2876.0
4,31001.0,2804.0,5680.0
5,31001.0,2686.0,8366.0
6,31001.0,3624.0,15588.0
7,31001.0,3598.0,15588.0
8,31001.0,1426.0,17014.0
9,31001.0,1103.0,18117.0


In [34]:
%%sql
SELECT start_terminal,
    duration_seconds,
    SUM(duration_seconds) OVER (
        PARTITION BY start_terminal
        ORDER BY start_time
    ) AS running_total,
    COUNT(duration_seconds) OVER (
        PARTITION BY start_terminal
        ORDER BY start_time
    ) AS running_count,
    AVG(duration_seconds) OVER (
        PARTITION BY start_terminal
        ORDER BY start_time
    ) AS running_avg,
    row_number() OVER (
        PARTITION BY start_terminal
        ORDER BY start_time
    ) AS row_numb,
    rank() OVER (
        PARTITION BY start_terminal
        ORDER BY start_time
    ) AS Rank_,
    lag(duration_seconds,1)OVER (
        PARTITION BY start_terminal
        ORDER BY start_time
    ) AS lag_,
    lead(duration_seconds,2)OVER (
        PARTITION BY start_terminal
        ORDER BY start_time
    ) AS lead_
FROM dc
WHERE start_time < '2012-01-08'
LIMIT 10

,start_terminal,duration_seconds,running_total,running_count,running_avg,row_numb,Rank_,lag_,lead_
0,31000.0,1422.0,1422.0,1,1422.000000,1,1,NaN,2661.0
1,31000.0,3340.0,4762.0,2,2381.000000,2,2,1422.0,NaN
2,31000.0,2661.0,7423.0,3,2474.333333,3,3,3340.0,NaN
3,31001.0,2876.0,2876.0,1,2876.000000,1,1,NaN,2686.0
4,31001.0,2804.0,5680.0,2,2840.000000,2,2,2876.0,3624.0
5,31001.0,2686.0,8366.0,3,2788.666667,3,3,2804.0,3598.0
6,31001.0,3624.0,15588.0,5,3117.600000,4,4,2686.0,1426.0
7,31001.0,3598.0,15588.0,5,3117.600000,5,4,3624.0,1103.0
8,31001.0,1426.0,17014.0,6,2835.666667,6,6,3598.0,1747.0
9,31001.0,1103.0,18117.0,7,2588.142857,7,7,1426.0,1740.0


In [36]:
%%sql
SELECT *
FROM (
        SELECT start_terminal,
            duration_seconds,
            duration_seconds - LAG(duration_seconds, 1) OVER (
                PARTITION BY start_terminal
                ORDER BY duration_seconds
            ) AS 'difference'
        FROM dc
        WHERE start_time < '2012-01-08'
        ORDER BY start_terminal,
            duration_seconds
    ) sub
WHERE sub.difference IS NOT NULL
LIMIT 10

,start_terminal,duration_seconds,difference
0,31000.0,2661.0,1239.0
1,31000.0,3340.0,679.0
2,31001.0,1157.0,54.0
3,31001.0,1168.0,11.0
4,31001.0,1264.0,96.0
5,31001.0,1426.0,162.0
6,31001.0,1740.0,314.0
7,31001.0,1747.0,7.0
8,31001.0,2686.0,939.0
9,31001.0,2804.0,118.0


In [41]:
%%sql
EXPLAIN format = json
select * from dc 
where duration_seconds <3000
limit 10

,EXPLAIN
0,"{\n ""query_block"": {\n ""select_id"": 1,\n ..."


In [45]:
res= %sql EXPLAIN FORMAT=json (select * from dc where duration_seconds <3000 limit 10)


                                             EXPLAIN
0  {\n  "query_block": {\n    "select_id": 1,\n  ...


In [48]:
res.to_csv("Explain.csv")